In [2]:

import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from PIL import Image
import os
from keras.utils import to_categorical
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'sklearn'

In [2]:
mapDict = {0: 0, 3: 1, 5: 2, 6: 3, 7: 4}

X_train = []
y_train = []
folder_path = 'train'

for foldername in os.listdir(folder_path):
    for filename in os.listdir(os.path.join(folder_path, foldername)):

        flattened = np.array(Image.open(os.path.join(folder_path, foldername, filename))).flatten()
        flattened[flattened > 0] = 1
        X_train.append(flattened)
        y_train.append(int(foldername))

X_train = np.array(X_train)
y_train = np.array(y_train)
for i, n in enumerate(y_train):
    y_train[i] = mapDict[y_train[i]]
y_train = to_categorical(y_train, num_classes=5)

X_test = []
y_test = []

folder_path = 'test'

for foldername in os.listdir(folder_path):
    for filename in os.listdir(os.path.join(folder_path, foldername)):

        flattened = np.array(Image.open(os.path.join(folder_path, foldername, filename))).flatten()
        flattened[flattened > 0] = 1
        X_test.append(flattened)
        y_test.append(int(foldername))

X_test = np.array(X_test)
y_test = np.array(y_test)
for i, n in enumerate(y_test):
    y_test[i] = mapDict[y_test[i]]
y_test = to_categorical(y_test, num_classes=5)


X_val = []
y_val = []


folder_path = 'val'

for foldername in os.listdir(folder_path):
    for filename in os.listdir(os.path.join(folder_path, foldername)):

        flattened = np.array(Image.open(os.path.join(folder_path, foldername, filename))).flatten()
        flattened[flattened > 0] = 1
        X_val.append(flattened)
        y_val.append(int(foldername))

X_val = np.array(X_val)
y_val = np.array(y_val)
for i, n in enumerate(y_val):
    y_val[i] = mapDict[y_val[i]]
y_val = to_categorical(y_val, num_classes=5)





In [3]:
model = keras.Sequential([
        layers.Dense(1000, activation="sigmoid", name="layer1"),
        layers.Dense(500, activation="sigmoid", name="layer2"),
        layers.Dense(100, activation="sigmoid", name="layer3"),
        layers.Dense(5, activation="softmax", name="output"),
        ])
model.build((None, 784))
initial_weights = model.get_weights()

In [4]:
sgd_optimizer = SGD(learning_rate=0.001,name='SGD') #Normal Gradient Descent
momentum_optimizer = SGD(learning_rate=0.001, momentum=0.9, name='Momentum_SGD') #Momentum Based
nag_optimizer = SGD(learning_rate=0.001, momentum=0.9, nesterov=True, name='NAG') #NAG
rms_optimizer = RMSprop(learning_rate=0.001, rho=0.99, momentum=0.0, epsilon=1e-08, name="RMSProp") #RMSProp
adagrad_optimizer = Adagrad(learning_rate=0.001, epsilon=1e-08, name="Adagrad") #AdaGrad
adam_optimizer = Adam(learning_rate=0.001) #Adam

In [5]:
# model = keras.models.load_model("model.34-0.15.h5")
class AverageLossCallback(Callback):
   def __init__(self, patience=2, threshold=0.0005):
      super().__init__()
      self.patience = patience
      self.threshold = threshold
      self.best_loss = np.Inf
      self.wait = 0
      self.losses = []

   def on_epoch_end(self, epoch, logs=None):
      if logs is None:
         logs = {}
      train_loss = logs.get('loss')
      if train_loss is None:
         return
      self.losses.append(train_loss)
      if len(self.losses) > self.patience:
         cur_loss = np.mean(self.losses[-self.patience:])
         prev_loss = np.mean(self.losses[-self.patience-1:-1])
         if abs(cur_loss - prev_loss) < self.threshold:
               self.wait += 1
               if self.wait >= self.patience:
                  self.model.stop_training = True
         else:
               self.wait = 0
               
average_loss_callback = AverageLossCallback()
my_callbacks = [
   average_loss_callback]

# ,
#    ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
#    TensorBoard(log_dir='./logs')

In [6]:
num_epochs_dict = {} # number of epochs each optimizer took till convergence
loss_dict = {} # loss at each epoch for each optimizer
eval_dict = {} # (train_accuracy, val_accuracy) for each optimizer

In [8]:
# SGD Optimizer
model.set_weights(initial_weights)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1000, verbose="auto", callbacks=[
   average_loss_callback,
   ModelCheckpoint(filepath=f"model_checkpoint_{model.optimizer.get_config()['name']}.h5",
                   save_best_only=True,  # Only save the best performing model
                    monitor='loss',  # Monitor the validation loss
                    mode='min',          # Minimize the validation loss
                    save_weights_only=False, # Save the entire model (including optimizer state)
                    append_history=True,),
   TensorBoard(log_dir='./logs')],
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['SGD'] = num_epochs
loss_dict['SGD'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['SGD'] = (train_accuracy, val_accuracy) 

with open('model_history_sgd.pkl', 'wb') as f:
    pickle.dump(model_fit.history, f)

Epoch 1/1000
 3156/11385 [=======>......................] - ETA: 41s - loss: 0.5122 - accuracy: 0.2031

KeyboardInterrupt: 

In [ ]:
# Vanilla Optimizer
model.set_weights(initial_weights)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=sgd_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=len(y_train), epochs=1000, verbose="auto", callbacks=my_callbacks,
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['VGD'] = num_epochs
loss_dict['VGD'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['VGD'] = (train_accuracy, val_accuracy) 

with open('model_history_vgd.pkl', 'wb') as f:
    pickle.dump(model_fit.history, f)

In [ ]:
# SGD with momentum (generalized delta rule)
model.set_weights(initial_weights)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=momentum_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1000, verbose="auto", callbacks=[
   average_loss_callback,
   ModelCheckpoint(filepath=f"model_checkpoint_{model.optimizer.get_config()['name']}.h5",
                   save_best_only=True,  # Only save the best performing model
                    monitor='loss',  # Monitor the validation loss
                    mode='min',          # Minimize the validation loss
                    save_weights_only=False, # Save the entire model (including optimizer state)
                    append_history=True,),
   TensorBoard(log_dir='./logs')],
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['momentum'] = num_epochs
loss_dict['momentum'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['momentum'] = (train_accuracy, val_accuracy) 

In [ ]:
# SGD with momentum (NAG)
model.set_weights(initial_weights)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=nag_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1000, verbose="auto", callbacks=[
   average_loss_callback,
   ModelCheckpoint(filepath=f"model_checkpoint_{model.optimizer.get_config()['name']}.h5",
                   save_best_only=True,  # Only save the best performing model
                    monitor='loss',  # Monitor the validation loss
                    mode='min',          # Minimize the validation loss
                    save_weights_only=False, # Save the entire model (including optimizer state)
                    append_history=True,),
   TensorBoard(log_dir='./logs')],
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['NAG'] = num_epochs
loss_dict['NAG'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['NAG'] = (train_accuracy, val_accuracy) 

In [ ]:
# AdaGrad
model.set_weights(initial_weights)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=adagrad_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1000, verbose="auto", callbacks=[
   average_loss_callback,
   ModelCheckpoint(filepath=f"model_checkpoint_{model.optimizer.get_config()['name']}.h5",
                   save_best_only=True,  # Only save the best performing model
                    monitor='loss',  # Monitor the validation loss
                    mode='min',          # Minimize the validation loss
                    save_weights_only=False, # Save the entire model (including optimizer state)
                    append_history=True,),
   TensorBoard(log_dir='./logs')],
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['adaGrad'] = num_epochs
loss_dict['adaGrad'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['adaGrad'] = (train_accuracy, val_accuracy) 

In [ ]:
# RMSProp
model.set_weights(initial_weights)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=rms_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1000, verbose="auto", callbacks=[
   average_loss_callback,
   ModelCheckpoint(filepath=f"model_checkpoint_{model.optimizer.get_config()['name']}.h5",
                   save_best_only=True,  # Only save the best performing model
                    monitor='loss',  # Monitor the validation loss
                    mode='min',          # Minimize the validation loss
                    save_weights_only=False, # Save the entire model (including optimizer state)
                    append_history=True,),
   TensorBoard(log_dir='./logs')],
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['rms'] = num_epochs
loss_dict['rms'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['rms'] = (train_accuracy, val_accuracy)  

In [ ]:
# Adam
model.set_weights(initial_weights)
# model = keras.models.load_model('model.14-0.10.h5')
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=adam_optimizer)
model_fit = model.fit(x=X_train, y=y_train, batch_size=1, epochs=1000, verbose="auto", callbacks=[
   average_loss_callback,
   ModelCheckpoint(filepath=f"model_checkpoint_{model.optimizer.get_config()['name']}.h5",
                   save_best_only=True,  # Only save the best performing model
                    monitor='loss',  # Monitor the validation loss
                    mode='min',          # Minimize the validation loss
                    save_weights_only=False, # Save the entire model (including optimizer state)
                    append_history=True,),
   TensorBoard(log_dir='./logs')],
    validation_split=0.0, validation_data=(X_val, y_val), shuffle=True, validation_batch_size=None)

train_loss_list = model_fit.history['loss']

num_epochs = len(model_fit.epoch)
num_epochs_dict['adam'] = num_epochs
loss_dict['adam'] = train_loss_list

train_accuracy, val_accuracy = model_fit.history['accuracy'][-1], model_fit.history['val_accuracy'][-1]

eval_dict['adam'] = (train_accuracy, val_accuracy) 

In [ ]:
# plot of number of epochs it took to train each optimizer

nums = []
names = []
for ele in num_epochs_dict:
    names.append(ele)
    nums.append(num_epochs_dict[ele])
data = pd.DataFrame({'num': nums, 'names': names})
ax = sns.barplot(x = names, y = nums)

# Set y-axis limits to integers
ax.set_yticks(range(int(min(nums)), int(max(nums))+1))

# Show the plot
ax.set_title('number of epochs it took to train each optimizer')
ax.set_xlabel('optimizer')
ax.set_ylabel('number of epochs to train')
plt.show()

In [ ]:
# plot of loss vs epoch for each optimizer
for opt in loss_dict:
    plt.plot([i for i in range(1, num_epochs_dict[opt]+1)], loss_dict[opt], label = opt)
plt.title('plots of average training error (y-axis) vs. epochs (x-axis) ')
plt.xlabel('epoch')
plt.ylabel('average training error')
plt.legend()
plt.show()

In [ ]:
# plot of accuracies of each optimizer
# Heights of the bars
opts = list(eval_dict.keys())
bar1 = (eval_dict[opt][0] for opt in opts)
bar2 = (eval_dict[opt][1] for opt in opts)

# Positions of the bars on the x-axis
x_pos = range(len(bar1))

# Set up the figure and axis objects
fig, ax = plt.subplots()

# Plot the bars
ax.bar(x_pos, bar1, width=0.4, color='b', label='training accuracy')
ax.bar([i + 0.4 for i in x_pos], bar2, width=0.4, color='g', label='validation accuracy')

# Set the x-axis labels and title
ax.set_xticks(x_pos)
ax.set_xticklabels(opts)
ax.set_xlabel('optimizer')
ax.set_ylabel('accuracy')
ax.set_title('plot of accuracies of each optimizer')

# Add a legend
ax.legend()

# Show the plot
plt.tight_layout()
plt.show()